<a href="https://colab.research.google.com/github/Evgeniya371/PRA3024/blob/main/Task_3_(Reflectance_of_the_mulitlayer_structure).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='orange'> Task 3


###Run each cell below one by one, read the annotations



*   Run this cell

*   The first two lines can be commented once you run the cell (spicy and pandas packages will be already imported into the environment)






In [17]:
# This cell imports all important packages

#!pip install scipy
#!pip install pandas
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib

# Functions


Run without changes





In [18]:
# This function computes a single film matrix
# s-polarization

def single_film_matrix_s(n, d, lambda_o, phi):
# n - refraction index of the layer
# d - thickness of the layer, expressed as a fraction of the refernece wavelength
# lambda_o - wavelength of the incident light
# phi - propagation angle (in degrees)

    phi = math.radians(phi)    # Conversion to radians
    k_0 = 2*np.pi/lambda_o     # Wave number: k=2*pi/wavelength
    optical_path=n*d*lambda_o  # Optical path = n * geometrical distance, where geometrical distnace = d * wavelength

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0*optical_path*np.cos(phi)), decimals=5)
    m_12 = - 1j /(n * np.cos(phi))* np.around(np.sin(k_0*optical_path*np.cos(phi)), decimals=5)
    m_21 = - 1j * n * np.cos(phi) * np.around(np.sin(k_0*optical_path*np.cos(phi)), decimals=5)
    m_22 = np.around(np.cos(k_0*optical_path*np.cos(phi)), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix=np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix


# This function computes a single film matrix
# p-polarization

def single_film_matrix_p(n, d, lambda_o, phi):
# n - refraction index of the layer
# d - thickness of the layer, expressed as a fraction of the refernece wavelength
# lambda_o -  wavelength of the incident light
# phi - propagation angle (in degrees)

    phi = math.radians(phi)    # Conversion to radians
    k_0 = 2*np.pi/lambda_o     # Wave number: k=2*pi/wavelength
    optical_path=n*d*lambda_o  # Optical path = n * geometrical distance, where geometrical distnace = d * wavelength

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0*optical_path*np.cos(phi)), decimals=5)
    m_12 = - 1j * n /np.cos(phi)* np.around(np.sin(k_0*optical_path*np.cos(phi)), decimals=5)
    m_21 = - 1j * np.cos(phi)/ n * np.around(np.sin(k_0*optical_path*np.cos(phi)), decimals=5)
    m_22 = np.around(np.cos(k_0*optical_path*np.cos(phi)), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix=np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix


# This function computes reflection coefficient
# s-polarization

def reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix

    angle1 = math.radians(phi_1)       # Conversion to radians
    angle_sub = math.radians(phi_sub)  # Conversion to radians

    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) - (m_21 + m_22 * n_sub * np.cos(phi_sub)))/((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) + m_21 + m_22 * n_sub * np.cos(phi_sub))

    return r


# This function computes reflection coefficient
# p-polarization

def reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix

    angle1 = math.radians(phi_1)        # Conversion to radians
    angle_sub = math.radians(phi_sub)   # Conversion to radians

    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * np.cos(phi_sub)/n_sub) * np.cos(phi_1)/n_1 - (m_21 + m_22 * np.cos(phi_sub)/n_sub))/((m_11 + m_12 * np.cos(angle_sub)/n_sub) * np.cos(phi_1)/n_1 + m_21 + m_22 * np.cos(phi_sub)/n_sub)

    return r


# This function computes the M_stack matrix and reflection coefficient of the multilayer coating

def stack(layers_dict, lambda_o, phi_1, phi_sub, n_1, n_sub, polarization):
# layers_dict - dictionary that represent a stack of coating layers
# lambda_o - wavelength of the incident light
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# polarization - polarization of the light wave ('s' or 'p')

    M_stack = np.identity(2) # Initialization of the M_stack matrix. Identity matrix is chosen as the intial matrix, since multiplication by identity matrix does not change the mutliplied matrix.

    # Iteration over the supplied layers_dict dictionary
    for k, v in layers_dict.items():
        refractive_index = v['refractive index']  # Select refractive index
        thickness = v['thickness']                # Select thickness
        angle = v['angle']                        # Select angle


        # If input value is 's', compute s-polarization
        if polarization == 's':
           M = single_film_matrix_s(n = refractive_index, d = thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matric multiplication for the successive layers
           r = reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M_stack)

        # If input value is 'p', compute p-polarization
        if polarization == 'p':
           M = single_film_matrix_p(n = refractive_index, d = thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matric multiplication for the successive layers
           r = reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M_stack)

    return M_stack, r



*   Fill in the empty spaces in this cell. Namely, specify the parameter values and define the multilayer structure
*   Run

**Examples of the implementation are provided in the next cells.**

In [ ]:
# Input values
angle_1 =       # Propagation angle in the incidence medium (usually air)
angle_sub =     # Propagation angle in the substrate
refraction_index_1 =      # Refraction index of the incidence medium (usually air)
refraction_index_sub =   # Refraction index of the substrate material
polarization = ' '     # Select the polarization by specifing the input as 's' or 'p'
wavelength =     # Wavelength of the incident light

# Define a stack of coating layers

layers = {

    'layer 1': {
         'thickness': ,           # Thickness of the layer (expressed as a fraction of the wavelength!!!)
         'refractive index': ,    # Refractive index of the layer
         'angle': ,               # Propagation angle in the layer (in degrees)
         },
    'layer 2': {
         'thickness': ,
         'refractive index': ,
         'angle': ,
         },
     'layer 3': {
         'thickness': ,
         'refractive index': ,
         'angle': ,
         }
     ### ...........
     ### ...........
     ### ...........
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M, r = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, phi_sub = angle_sub, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M)
print('Reflection coefficient: ', r)

## Example: 3 QW layers

Polarization: s

Structure: incidence medium|LHH|substrate

In [19]:
# Input values
angle_1 = 0      # Propagation angle in the incidence medium (usually air)
angle_sub = 0    # Propagation angle in the substrate
refraction_index_1 = 1      # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.2  # Refraction index of the substrate material
polarization = 's'     # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800e-9    # Wavelength of the incident light

# Define a stack of coating layers

layers = {

    'layer 1': {
         'thickness': 0.125,       # Thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,    # Refractive index of the layer
         'angle': 0,               # Propagation angle in the layer (in degrees)
         },
    'layer 2': {
         'thickness': 0.0625,
         'refractive index': 4,
         'angle': 0,
         },
     'layer 3': {
         'thickness': 0.0625,
         'refractive index': 4,
         'angle': 0,
         }
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M, r = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, phi_sub = angle_sub, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M)
print('Reflection coefficient: ', r)


Matrix, stack:
[[0j 0.5j]
 [2j 0j]]
Reflection coefficient:  (-0.5384615384615383-0j)


## Example: 1 QW layer

Polarization: p

Structure: incidence medium|layer|substrate

In [20]:
# Input values
angle_1 = 0     # Propagation angle in the incidence medium (usually air)
angle_sub = 0   # Propagation angle in the substrate
refraction_index_1 = 1      # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.2  # Refraction index of the substrate material
polarization = 'p'    # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800e-9   # Wavelength of the incident light

# Define a stack of coating layers
layers = {

    'layer 1': {
         'thickness': 0.125,      # Thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,   # Refractive index of the layer
         'angle': 0,              # Propagation angle in the layer (in degrees)
         },
    }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M, r = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, phi_sub = angle_sub, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M)
print('Reflection coefficient: ', r)

Matrix, stack:
[[0j -2j]
 [-0.5j 0j]]
Reflection coefficient:  (0.5384615384615384-0j)


# Example: arbitrary thickness of layers

Polarization: s

Structure: incidence medium|layer_1, layer_2, layer_3, layer_4|substrate

In [21]:
# Input values
angle_1 = 0      # Propagation angle in the incidence medium (usually air)
angle_sub = 0    # Propagation angle in the substrate
refraction_index_1 = 1      # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.2  # Refraction index of the substrate material
polarization = 's'     # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800e-9    # Wavelength of the incident light

# Define a stack of coating layers

layers = {

    'layer 1': {
         'thickness': 0.1,         # Thickness of the layer (expressed as a fraction of the wavelength)
         'refractive index': 2,    # Refractive index of the layer
         'angle': 0,               # Propagation angle in the layer (in degrees)
         },
    'layer 2': {
         'thickness': 0.25,
         'refractive index': 4,
         'angle': 0,
         },
     'layer 3': {
         'thickness': 0.2,
         'refractive index': 1.4,
         'angle': 0,
         },
    'layer 4': {
         'thickness': 0.4,
         'refractive index': 1.1,
         'angle': 0,
         }
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M, r = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, phi_sub = angle_sub, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M)
print('Reflection coefficient: ', r)


Matrix, stack:
[[(0.610153605612615-0j) 0.356972338684983j]
 [0.6276011528025014j (1.2717786422451092-0j)]]
Reflection coefficient:  (-0.3816274253996555+0.09537664840256957j)
